In [ ]:
from sklearn.neural_network import MLPClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import joblib
import pickle
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras import Sequential,layers, losses, optimizers
from tensorflow.keras.layers import Dense
import tensorflow as tf
import time

In [ ]:
df_train = pd.read_csv("Malicious_Packet_Detector_Training.csv")

In [ ]:
df_test = pd.read_csv("Malicious_Packet_Detector_Testing.csv")

In [ ]:
infiltration_attack = df_test[df_test['Label'] == 'Infiltration'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(infiltration_attack.shape)
portscan_attack = df_test[df_test['Label'] == 'Port Scan'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(portscan_attack.shape)
ddos_attack = df_test[df_test['Label'] == 'DDoS'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(ddos_attack.shape)
bruteforce_attack = df_test[df_test['Label'] == 'Brute Force'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(bruteforce_attack.shape)
dos_attack = df_test[df_test['Label'] == 'DoS'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(dos_attack.shape)
web_attack = df_test[df_test['Label'] == 'Web Attack'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(web_attack.shape)
benign_data = df_test[df_test['Label'] == 'Benign'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(benign_data.shape)

In [ ]:
df_test_2018 = pd.read_csv("Malicious_Packet_Detector_Testing_2018.csv")

In [ ]:
df_test_2018 = df_test_2018.sample(frac = 1).reset_index(drop = True)

In [ ]:
df_test_2018

In [ ]:
X_train_2018 = df_test_2018.iloc[:,0:1525]
Y_train_2018 = df_test_2018.iloc[:,1526:1527]

In [ ]:
X_train_2017 = df_test.iloc[:,0:1525]
Y_train_2017 = df_test.iloc[:,1526:1527]

In [ ]:
Y_train_2017.loc[Y_train_2017['Label_binary'] == 'Benign', 'Label_binary'] = 0
Y_train_2017.loc[Y_train_2017['Label_binary'] == 'Malicious', 'Label_binary'] = 1

In [ ]:
Y_train_2018.loc[Y_train_2018['Label_binary'] == 'Benign', 'Label_binary'] = 0
Y_train_2018.loc[Y_train_2018['Label_binary'] == 'Malicious', 'Label_binary'] = 1

In [ ]:
X_train_total = pd.concat([X_train_2017,X_train_2018])
Y_train_total = pd.concat([Y_train_2017,Y_train_2018])

In [ ]:
X_train_ = X_train_total.values.astype('float32')
Y_train_ = Y_train_total.values.astype('float32')

In [ ]:
infiltration_attack_2018 = df_test_2018[df_test_2018['Label'] == 'Infiltration'].drop(['Label_binary','Label'], axis=1).iloc[:,0:500]
print(infiltration_attack_2018.shape)
portscan_attack_2018 = df_test_2018[df_test_2018['Label'] == 'Port Scan'].drop(['Label_binary','Label'], axis=1).iloc[:,0:500]
print(portscan_attack_2018.shape)
ddos_attack_2018 = df_test_2018[df_test_2018['Label'] == 'ddos'].drop(['Label_binary','Label'], axis=1).iloc[:,0:500]
print(ddos_attack_2018.shape)
bruteforce_attack_2018 = df_test_2018[df_test_2018['Label'] == 'Brute Force'].drop(['Label_binary','Label'], axis=1).iloc[:,0:500]
print(bruteforce_attack_2018.shape)
dos_attack_2018 = df_test_2018[df_test_2018['Label'] == 'DoS'].drop(['Label_binary','Label'], axis=1).iloc[:,0:500]
print(dos_attack_2018.shape)
web_attack_2018 = df_test_2018[df_test_2018['Label'] == 'Web Attack'].drop(['Label_binary','Label'], axis=1).iloc[:,0:500]
print(web_attack_2018.shape)
benign_data_2018 = df_test_2018[df_test_2018['Label'] == 'Benign'].drop(['Label_binary','Label'], axis=1).iloc[:,0:500]
print(benign_data_2018.shape)

In [ ]:
stage3_AE = tf.keras.models.load_model('Novelty_Detector_Baseline.h5', compile = False)

In [ ]:
def recon_metrices(data, reconstructed_data):

    maes = np.absolute(data.values - reconstructed_data)
    
    for j in range(len(maes)):
        m = maes[j]
        d = data.values[j]
        r = reconstructed_data[j]
        for el in range(len(m)):
            if r[el] == 0 or d[el] == 0:
                m[el] = 0
 
            
    mae_byte_list = []
    ip_mae_list = []
    tcp_header_mae_list = []
    tcp_options_mae_list = []
    tcp_segment_mae_list = []
    i=0
    for mae in maes:
#         print(np.count_nonzero(reconstructed_data[i]))
#         print(np.count_nonzero(data.values[i]))
#         print(np.max(np.count_nonzero(reconstructed_data[i]),np.count_nonzero(data.values[i])))
        mae_byte = sum(mae) / np.count_nonzero(mae)
        mae_byte_list.append(mae_byte)
        i += 1
    return mae_byte_list

In [ ]:
bot = pd.read_csv('botnet_test_normalized.csv')
bot_attack_2017 = bot[bot['Label'] == 'Botnet'].drop(['Label'], axis=1).iloc[:,0:500]
print(bot_attack_2017.shape)

In [ ]:
bot = stage3_AE.predict(bot_attack_2017)

In [ ]:
mae_byte_list = recon_metrices(bot_attack_2017,bot)

In [ ]:
## Classification
count_99 = len([i for i in mae_byte_list if i > 0.07])
print(count_99 / len(mae_byte_list))

In [ ]:
inf = stage3_AE.predict(infiltration_attack_2018)

In [ ]:
mae_byte_list = recon_metrices(infiltration_attack_2018,inf)

In [ ]:
## Classification
count_99 = len([i for i in mae_byte_list if i > 0.09])
print(count_99 / len(mae_byte_list))

In [ ]:
stage3_AE.summary()

In [ ]:
### Retraining with CICIDS 2018 data

In [ ]:
mod_layers = [5,6,8,9]

In [ ]:
layer5_ind = [h for h in range(0,250,3)]
layer6_ind = [h for h in range(0,125,3)]
layer8_ind = [h for h in range(0,125,3)]
layer9_ind = [h for h in range(0,250,3)]

In [ ]:
retrained_stage3 = tf.keras.Sequential()

In [ ]:
for i, layer in enumerate(stage3_AE.layers):
    if i not in mod_layers:
        layer.trainable = False
        retrained_stage3.add(layer)
    else:
        retrained_stage3.add(layer)
        if i == 5:
            for ind in range(0,250):
                if ind in layer5_ind:
                    retrained_stage3.layers[-1].kernel[ind]._trainable = False
        if i == 6:
            for ind in range(0,125):
                if ind in layer6_ind:
                    retrained_stage3.layers[-1].kernel[ind]._trainable = False
                    
        if i == 8:
            for ind in range(0,62):
                if ind in layer8_ind:
                    retrained_stage3.layers[-1].kernel[ind]._trainable = False
        
        if i == 9:
            for ind in range(0,125):
                if ind in layer9_ind:
                    retrained_stage3.layers[-1].kernel[ind]._trainable = False

In [ ]:
retrained_stage3.summary()

In [ ]:
df_adv = pd.read_csv("adv_examples_all_allDNN_bigger.csv")
df_adv_syn = pd.read_csv("Adv_all_DNN_incremental.csv")

In [ ]:
df_adv_syn= df_adv_syn.loc[:, ~df_adv_syn.columns.str.contains('^Unnamed')]

In [ ]:
adv_train = df_adv.iloc[:,:500]

In [ ]:
adv_syn = df_adv_syn.iloc[:,:500]

In [ ]:
adv_total = pd.concat([adv_train,adv_syn])

In [ ]:
start_time = time.time()
retrained_stage3.compile(optimizer='adam', loss='mae')
retrained_stage3.fit(adv_total , adv_total , batch_size=256, epochs=500, shuffle=True)
end_time = time.time()

In [ ]:
retrained_stage3.save('Retrained_Novelty_Detector.h5')

In [ ]:
adv_test = retrained_stage3.predict(adv_syn)

In [ ]:
time_taken = end_time - start_time
print(time_taken)

In [ ]:
mae_byte_list = recon_metrices(adv_syn,adv_test)

In [ ]:
mae_byte_avg = sum(mae_byte_list) / len(mae_byte_list)
print('mae_byte_avg', mae_byte_avg)
mae_byte_per_95 = np.percentile(mae_byte_list, 99)
print('mae_byte_99_percentile', mae_byte_per_95)
mae_byte_per_95 = np.percentile(mae_byte_list, 95)
print('mae_byte_95_percentile', mae_byte_per_95)
mae_byte_per_95 = np.percentile(mae_byte_list, 90)
print('mae_byte_90_percentile', mae_byte_per_95)

In [ ]:
## CICIDS2017 and CICIDS2018 Testing retrained model

In [ ]:
inf_ = retrained_stage2.predict(infiltration_attack)
port_ = retrained_stage2.predict(portscan_attack)
dos_ = retrained_stage2.predict(dos_attack)
ddos_ = retrained_stage2.predict(ddos_attack)
web_ = retrained_stage2.predict(web_attack)
brute_ = retrained_stage2.predict(bruteforce_attack)
ben_ = retrained_stage2.predict(benign_data)

In [ ]:
mae_byte_list = recon_metrices(benign_data,ben_)

In [ ]:
mae_byte_avg = sum(mae_byte_list) / len(mae_byte_list)
print('mae_byte_avg', mae_byte_avg)
mae_byte_per_95 = np.percentile(mae_byte_list, 99)
print('mae_byte_99_percentile', mae_byte_per_95)
mae_byte_per_95 = np.percentile(mae_byte_list, 95)
print('mae_byte_95_percentile', mae_byte_per_95)
mae_byte_per_95 = np.percentile(mae_byte_list, 90)
print('mae_byte_90_percentile', mae_byte_per_95)

In [ ]:
inf_2018 = retrained_stage3.predict(infiltration_attack_2018)
dos_2018 = retrained_stage3.predict(dos_attack_2018)
ddos_2018 = retrained_stage3.predict(ddos_attack_2018)
web_2018 = retrained_stage3.predict(web_attack_2018)
brute_2018 = retrained_stage3.predict(bruteforce_attack_2018)
ben_2018 = retrained_stage3.predict(benign_data_2018)

In [ ]:
mae_byte_list = recon_metrices(infiltration_attack_2018, inf_2018)

In [ ]:
## Classification
count_99 = len([i for i in mae_byte_list if i > 0.10])
print(count_99 / len(mae_byte_list))

In [ ]:
df_rand = pd.read_csv('Adv_all_DNN_truerandom.csv')
df_rand = df_rand.loc[:, ~df_rand.columns.str.contains('^Unnamed')]
adv_rand = df_rand.iloc[:,:500]

In [ ]:
rand_adv = retrained_stage3.predict(adv_train)
mae_byte_list = recon_metrices(adv_train, rand_adv)

In [ ]:
## Classification
count_99 = len([i for i in mae_byte_list if i > 0.10])
print(count_99 / len(mae_byte_list))

In [ ]:
print('Infiltration')
cnt_inf, acc_inf = accuracy_calc(inf_2018)
print(cnt_inf)
print(acc_inf)
print('DoS')
cnt_dos, acc_dos = accuracy_calc(dos_2018)
print(cnt_dos)
print(acc_dos)
print('DDoS')
cnt_ddos, acc_ddos = accuracy_calc(ddos_2018)
print(cnt_ddos)
print(acc_ddos)
print('Web')
cnt_web, acc_web = accuracy_calc(web_2018)
print(cnt_web)
print(acc_web)
print('Brute')
cnt_brute, acc_brute = accuracy_calc(brute_2018)
print(cnt_brute)
print(acc_brute)
print('Benign')
cnt_ben, acc_ben = accuracy_calc(ben_2018)
print(cnt_ben)
print(1-acc_ben)

In [ ]:
## CICIDS2017 Testing Retrained Model

In [ ]:
inf = retrained_stage1.predict(infiltration_attack)
dos = retrained_stage1.predict(dos_attack)
ddos = retrained_stage1.predict(ddos_attack)
web = retrained_stage1.predict(web_attack)
brute = retrained_stage1.predict(bruteforce_attack)
ben = retrained_stage1.predict(benign_data)

In [ ]:
print('Infiltration')
cnt_inf, acc_inf = accuracy_calc(inf)
print(cnt_inf)
print(acc_inf)
print('DoS')
cnt_dos, acc_dos = accuracy_calc(dos)
print(cnt_dos)
print(acc_dos)
print('DDoS')
cnt_ddos, acc_ddos = accuracy_calc(ddos)
print(cnt_ddos)
print(acc_ddos)
print('Web')
cnt_web, acc_web = accuracy_calc(web)
print(cnt_web)
print(acc_web)
print('Brute')
cnt_brute, acc_brute = accuracy_calc(brute)
print(cnt_brute)
print(acc_brute)
print('Benign')
cnt_ben, acc_ben = accuracy_calc(ben)
print(cnt_ben)
print(1-acc_ben)